# **Part 1: Import, Load Data.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

from google.colab import drive
drive.mount('/content/drive')

checks = pd.read_csv('/content/drive/MyDrive/20210518_checks.csv')
hie = pd.read_csv('/content/20210518_hierarchy.csv')
uplift = pd.read_csv('/content/20210518_uplift.csv')
off = pd.read_csv('/content/20210521_offers.csv')


Mounted at /content/drive



# **Part 2: Exploratory Data Analysis.**

In [ ]:
checks.describe()

,day,check_pos,num_sales,supplier_price,selling_price
count,2.147926e+07,2.147926e+07,2.147926e+07,2.147926e+07,2.147926e+07
mean,2.019775e+07,1.200852e+01,1.480220e+00,8.424017e+01,1.275595e+02
std,4.274481e+03,1.200505e+01,1.929495e+00,1.496848e+02,2.175990e+02
min,2.019092e+07,1.000000e+00,2.000000e-03,0.000000e+00,0.000000e+00
25%,2.019122e+07,4.000000e+00,1.000000e+00,2.549000e+01,4.249000e+01
50%,2.020032e+07,8.000000e+00,1.000000e+00,4.841000e+01,7.789000e+01
75%,2.020062e+07,1.600000e+01,1.560000e+00,9.598000e+01,1.459900e+02
max,2.020092e+07,2.920000e+02,2.009000e+03,4.417559e+04,5.836344e+04


In [ ]:
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21479259 entries, 0 to 21479258
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   client_id       object 
 1   day             int64  
 2   shop_id         object 
 3   check_id        object 
 4   time            object 
 5   sku             object 
 6   promo_id        object 
 7   check_pos       int64  
 8   num_sales       float64
 9   supplier_price  float64
 10  selling_price   float64
 11  region_name     object 
dtypes: float64(3), int64(2), object(7)
memory usage: 1.9+ GB


In [ ]:
#сколько товаров в каждом offer_id в среднем
offer_group = off.groupby('Offer_ID').count()['sku'].mean()
print(offer_group)


10.996625421822273


In [ ]:
off.query('Offer_ID == "F97"')

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
2176,Facades,F97,0d093fdb80eaad391f98db998d1f1b5e,20200618,20200624,train
2382,Facades,F97,55ab9a9c4f7e8262ec600218304f50a6,20200618,20200624,train
2389,Facades,F97,d9b38fad6052a1c1045b18b67f34978b,20200618,20200624,train
2454,Facades,F97,1775f1b15adc87f37570b3aca62b29c5,20200618,20200624,train
2509,Facades,F97,0eb4404a047fc38886b633d6fb8405a6,20200618,20200624,train
2596,Facades,F97,9794c588b279cd37153e81d9109acf70,20200618,20200624,train
2619,Facades,F97,5cc9ca284d9fad6d306fd04555eec65d,20200618,20200624,train
2664,Facades,F97,21f1dee058ef497125efb844c7808862,20200618,20200624,train
2678,Facades,F97,60551c0347fa6c64066d2052dad317c0,20200618,20200624,train
2687,Facades,F97,20d7916bda0cc81d967ad01bfcb36bfa,20200618,20200624,train


In [ ]:
checks

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name
0,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,1,5.0,133.40,249.45,152f1b77a32508570e2745daf9ce7aec
1,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.0,96.77,118.99,152f1b77a32508570e2745daf9ce7aec
2,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,5,2.0,2.34,7.58,152f1b77a32508570e2745daf9ce7aec
3,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,4,1.0,23.59,39.99,152f1b77a32508570e2745daf9ce7aec
4,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,3,1.0,24.02,39.99,152f1b77a32508570e2745daf9ce7aec
...,...,...,...,...,...,...,...,...,...,...,...,...
21479254,128bfb96146f0772ce9ce35365b5b402,20200101,087c8abfaee44ebbf0c2871976a2ab18,ac8cc7468ed294fcecebdcfa00d4280f,21:10:08,dc155ba1e8a3c759bcf2f358384ba9cd,f1b40313f0dc51fde3fe2cc7bc2b987f,10,1.0,63.63,99.89,7e35e74e610188414ad24235dd787c78
21479255,128bfb96146f0772ce9ce35365b5b402,20200101,087c8abfaee44ebbf0c2871976a2ab18,ac8cc7468ed294fcecebdcfa00d4280f,21:10:08,305d39a0a0af37b5436e8a3ff3e9d319,357f92d49be136644805a94cdbeb462a,6,1.0,185.70,298.99,7e35e74e610188414ad24235dd787c78
21479256,128bfb96146f0772ce9ce35365b5b402,20200101,087c8abfaee44ebbf0c2871976a2ab18,ac8cc7468ed294fcecebdcfa00d4280f,21:10:08,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,2,1.0,1.14,3.49,7e35e74e610188414ad24235dd787c78
21479257,128bfb96146f0772ce9ce35365b5b402,20200101,087c8abfaee44ebbf0c2871976a2ab18,ac8cc7468ed294fcecebdcfa00d4280f,21:10:08,6540ad4c3e0b21d5a804b692dda6fe66,7215ee9c7d9dc229d2921a40e899ec5f,4,1.0,37.63,74.99,7e35e74e610188414ad24235dd787c78


In [ ]:
off.start_date = pd.to_datetime(off.start_date.map(lambda x: str(x)[:4]+ '-' + str(x)[4:6]+'-' +str(x)[6:]))
off.end_date = pd.to_datetime(off.end_date.map(lambda x: str(x)[:4]+ '-' + str(x)[4:6]+'-' +str(x)[6:]))
checks.day = pd.to_datetime(checks.day.map(lambda x: str(x)[:4]+ '-' + str(x)[4:6]+'-' +str(x)[6:]))
checks = checks.drop('client_id', 1)
checks = checks.drop('shop_id', 1)
checks = checks.drop('check_id', 1)
checks = checks.drop('region_name', 1)
checks = checks.drop('check_pos', 1)
checks = checks.drop('promo_id', 1)
checks = checks.merge(off, on='sku')
checks

,day,time,sku,num_sales,supplier_price,selling_price,Promo_type,Offer_ID,start_date,end_date,train_test_group
0,2019-12-30,07:28:39,b9ef00e971d0fb459a765708efba70c0,1.0,96.77,118.99,Biweekly,287,2020-03-19,2020-04-01,train
1,2019-12-30,07:28:39,b9ef00e971d0fb459a765708efba70c0,1.0,96.77,118.99,Biweekly,369,2020-06-25,2020-07-08,train
2,2019-12-30,15:58:46,b9ef00e971d0fb459a765708efba70c0,1.0,96.77,118.99,Biweekly,287,2020-03-19,2020-04-01,train
3,2019-12-30,15:58:46,b9ef00e971d0fb459a765708efba70c0,1.0,96.77,118.99,Biweekly,369,2020-06-25,2020-07-08,train
4,2019-12-30,11:12:14,b9ef00e971d0fb459a765708efba70c0,1.0,96.77,118.99,Biweekly,287,2020-03-19,2020-04-01,train
...,...,...,...,...,...,...,...,...,...,...,...
17821451,2019-12-03,13:43:06,5a7611f54e2ba26fe06f3f3c12ef3b02,1.0,14.40,14.89,Biweekly,68,2020-03-05,2020-03-18,train
17821452,2019-12-03,13:43:06,5a7611f54e2ba26fe06f3f3c12ef3b02,1.0,14.40,14.89,Biweekly,372,2020-11-26,2020-12-16,test
17821453,2020-08-11,12:47:24,8964706878d0b9b204714280296925d7,1.0,108.63,199.00,Biweekly,368,2020-06-25,2020-07-08,train
17821454,2020-06-25,21:31:57,3a8e820bc89e71dbb6a42eac4aeea9b1,1.0,754.60,1298.19,Seasonal,585,2019-11-05,2019-11-25,train


In [ ]:
checks = checks.query('day >= start_date & day <=end_date')
checks['profit'] = checks['selling_price']-checks['supplier_price']
checks_1qua_big_profit = checks.query('profit>=profit.median() & "2020-01-01" <= start_date & "2020-03-31" >= end_date')
checks_1qua =checks.query('"2020-01-01" <= start_date & "2020-03-31" >= end_date')
print("всего офферов за 1 квартал 2020: "+str(len(checks_1qua['Offer_ID'].unique())))
print("с профитом больше медианы: "+str(len(checks_1qua_big_profit['Offer_ID'].unique())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


всего офферов за 1 квартал 2020: 195
с профитом больше медианы: 176


In [ ]:
hie

,sku,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,29285fb39582e5a63c14026198c6ad51,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,f50c7035e532c49a0f6993d988e2e843,e9f41ed96e63ca1be6aea099625afe6f
1,f31d0af494b191ef2bc8395db9c8366f,44a7b825383402ed937a15fb8a5bb12a,ad61ab143223efbc24c7d2583be69251,84e8ce7870f0eecd843366582bb95a28,b7fa186535a75c869bfaece622c72689
2,49edad6669ba09dffbf278b76c53cd9d,cc9284597026dcc021735c0970ca0c93,1c383cd30b7c298ab50293adfecb7b18,2a3d6d6cc4b5e77238c1fc1bb6cdd681,7e0e0c1fe174b2d2efbf5ae0e2bc3ee9
3,c5c632eb0f93a233ef6db2620a464ea0,44a7b825383402ed937a15fb8a5bb12a,d82c8d1619ad8176d665453cfb2e55f0,b27d5296bede63b1493a5d321d4e8092,ec2acf8ebfdb679d5fc87caadd98fb58
4,9d0c7880f16c7748e1e992b28655c67f,cc9284597026dcc021735c0970ca0c93,6ea9ab1baa0efb9e19094440c317e21b,379a7ba015d8bf1c70b8add2c287c6fa,5aec2478fac4d1ee912c794c343d036c
...,...,...,...,...,...
80401,33d0eb46ca492585ccf3ba2bcc19ed30,44a7b825383402ed937a15fb8a5bb12a,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,b02ece95d7615f1b007e29a316cd7292
80402,bb47f5bc8df7b92ae0600c7d85a567fd,cc9284597026dcc021735c0970ca0c93,a1d0c6e83f027327d8461063f4ac58a6,3335881e06d4d23091389226225e17c7,4028a78ad465aea9c6024f81ca22c78d
80403,e9311b17cd22904c2d6fee2e3c8b6d5f,44a7b825383402ed937a15fb8a5bb12a,ad61ab143223efbc24c7d2583be69251,dcc5c249e15c211f21e1da0f3ba66169,9b4166e30f69d57136493e52c97bc874
80404,15cafb345d5e95b3ae83f2cbb87e0f72,44a7b825383402ed937a15fb8a5bb12a,d2ddea18f00665ce8623e36bd4e3c7c5,a523426cc585745318d5f6d91a9c0706,846a6cd5242ad9c4e8eea0621a9cf252


In [ ]:
good_offers = checks_1qua_big_profit['Offer_ID'].unique()
calendar = off.query('Offer_ID in @good_offers')
calendar = calendar.drop('train_test_group', 1)
calendar = calendar.drop('Offer_ID', 1)
calendar_sku = calendar['sku'].unique()

In [ ]:
calendar1 = pd.read_csv('/content/result_1.csv')
calendar1

,Unnamed: 0,Promo_type,sku,start_date,end_date,category
0,0,Facades,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,a5771bce93e200c36f7cd9dfd0e5deaa
1,2,Facades,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,c16a5320fa475530d9583c34fd356ef5
2,3,Facades,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,33e75ff09dd601bbe69f351039152189
3,7,Facades,bfce8277f8dacd84c86bcec698f8288d,2020-01-23,2020-01-29,19ca14e7ea6328a42e0eb13d585e4c22
4,8,Facades,06b43c04dfadb2fca40fd5762c317d48,2020-02-06,2020-02-12,c16a5320fa475530d9583c34fd356ef5
...,...,...,...,...,...,...
1620,8330,Facades,d6077144c2911234d5f26aa341e89f2c,2020-01-02,2020-01-08,182be0c5cdcd5072bb1864cdee4d3d6e
1621,8333,Facades,f06cea06222d914c117278950692697f,2020-01-02,2020-01-08,c20ad4d76fe97759aa27a0c99bff6710
1622,8334,Facades,96bcd91e9487b562b7aa5ef443c7817b,2020-01-02,2020-01-08,c20ad4d76fe97759aa27a0c99bff6710
1623,8337,Facades,c2ab155181206e7c5cf25f8de2509f9b,2020-01-02,2020-01-08,c20ad4d76fe97759aa27a0c99bff6710


In [ ]:
cat2_calendar = calendar1['category'].unique()
cat2 = hie['hierarchy_level2'].unique()
cat2_needed = []
for x in cat2:
  if x not in cat2_calendar:
    cat2_needed.append(x)
cat2_needed

['9a1158154dfa42caddbd0694a4e9bdc8',
 'ad61ab143223efbc24c7d2583be69251',
 'e369853df766fa44e1ed0ff613f563bd',
 'ea5d2f1c4608232e07d3aa3d998e5135',
 '3295c76acbf4caaed33c36b1b5fc2cb1',
 'a684eceee76fc522773286a895bc8436',
 '4e732ced3463d06de0ca9a15b6153677',
 'c0c7c76d30bd3dcaefc96f40275bdc0a',
 '093f65e080a295f8076b1c5722a46aa2',
 '3416a75f4cea9109507cacd8e2f2aefc',
 '70efdf2ec9b086079795c442636b55fb',
 'fc490ca45c00b1249bbe3554a4fdf6fb',
 '02e74f10e0327ad868d138f2b4fdd6f0',
 '2838023a778dfaecdc212708f721b788',
 '9f61408e3afb633e50cdf1b20de6f466',
 '072b030ba126b2f4b2374f342be9ed44',
 '67c6a1e7ce56d3d6fa748ab6d9af3fd7',
 'aab3238922bcc25a6f606eb525ffdc56',
 'c74d97b01eae257e44aa9d5bade97baf',
 'a1d0c6e83f027327d8461063f4ac58a6',
 'd9d4f495e875a2e075a1a4a6e1b9770f',
 '3c59dc048e8850243be8079a5c74d079',
 '98f13708210194c475687be6106a3b84',
 '28dd2c7955ce926456240b2ff0100bde',
 'a2ef406e2c2351e0b9e80029c909242d',
 '642e92efb79421734881b53e1e1b18b6',
 '96a3be3cf272e017046d1b2674a52bd3',
 

In [ ]:
hie_needed = hie.query('hierarchy_level2 in @cat2_needed')
hie_needed.to_csv('hie_needed.csv')
!cp hie_needed.csv "drive/My Drive/"

In [ ]:
hie_needed = pd.read_csv('/content/hie_needed.csv')
sku_needed = hie_needed['sku'].head(209)

In [ ]:
sku_needed

0      29285fb39582e5a63c14026198c6ad51
1      f31d0af494b191ef2bc8395db9c8366f
2      b139209608aef030a343a4f178de818d
3      4ed8978312e29286e2f45220b4cbdd3e
4      90e335650f966274a872412ff34505b0
                     ...               
204    c0bf7c3cf2e2dd962af8ab6bfdf0335c
205    b612178bd9b97a83e12b43de67b1ccab
206    4605ab09566768f24e1b4c61806a9b70
207    5edd33decdbaa89b11155f98036d50e2
208    8382b3045aeab8848de66cdc33050d30
Name: sku, Length: 209, dtype: object